<a href="https://colab.research.google.com/github/rajitha-1701/Answer-Type-Prediction/blob/main/QA_Type_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_excel('Train.xlsx')
df_test = pd.read_excel('testFinal.xlsx')

In [2]:
data = pd.concat([df, df_test], axis=0)
data.head()

,id,question,qtype,Unnamed: 0
0,19719,What periodical literature does delta air line...,publication,NaN
1,15554,Who is the child of ranavalona i's husband?,person,NaN
2,974,Is it true jeff_bridges occupation lane chandl...,boolean,NaN
3,27610,Which is the operating income for qantas?,number,NaN
4,24488,Which cola starts with the letter p,soft drink,NaN


In [3]:
len(data)
print(len(df))
print(len(df_test))

18251
4571


In [4]:
data.isna().sum()

id                0
question         16
qtype             0
Unnamed: 0    18251
dtype: int64

In [5]:
data = data[data['question'].notna()]

In [6]:
data['labels'] = pd.factorize(data.qtype)[0] + 1

In [7]:
df = data.iloc[:18240,:]
df_test = data.iloc[18241:,:]

In [8]:
df.head()

,id,question,qtype,Unnamed: 0,labels
0,19719,What periodical literature does delta air line...,publication,NaN,1
1,15554,Who is the child of ranavalona i's husband?,person,NaN,2
2,974,Is it true jeff_bridges occupation lane chandl...,boolean,NaN,3
3,27610,Which is the operating income for qantas?,number,NaN,4
4,24488,Which cola starts with the letter p,soft drink,NaN,5


In [9]:
df_test.head()

,id,question,qtype,Unnamed: 0,labels
18241,4287,What was al green nominated for at the 37th to...,award,NaN,21
18242,19367,Is the number of points/goals/set scored in th...,boolean,NaN,3
18243,17014,What country is near lefkada?,country,NaN,106
18244,3427,What is the grinder model with the lowest soun...,type of manufactured good,NaN,387
18245,24262,What new religious movement starts with the le...,religious movement,NaN,605


In [10]:
df = df.iloc[: , 1:]
df_test = df_test.iloc[: , 1:]
df.head()

,question,qtype,Unnamed: 0,labels
0,What periodical literature does delta air line...,publication,NaN,1
1,Who is the child of ranavalona i's husband?,person,NaN,2
2,Is it true jeff_bridges occupation lane chandl...,boolean,NaN,3
3,Which is the operating income for qantas?,number,NaN,4
4,Which cola starts with the letter p,soft drink,NaN,5


In [11]:
df['question'] = df['question'].str.lower()
df_test['question'] = df_test['question'].str.lower()

In [12]:
df['qtype'].value_counts()

boolean                                        2137
string                                         2085
date                                           1301
number                                         1044
person                                          731
                                               ... 
French independent administrative authority       1
measurement scale                                 1
type of medical procedure                         1
promissory note                                   1
derivation                                        1
Name: qtype, Length: 1682, dtype: int64

In [13]:
df_test['qtype'].value_counts()

boolean                                               532
string                                                502
date                                                  327
number                                                277
omnivore                                              276
                                                     ... 
web-safe color                                          1
maritime disaster                                       1
fictional dog                                           1
angel in Judaism                                        1
colleges within universities in the United Kingdom      1
Name: qtype, Length: 769, dtype: int64

In [14]:
df['labels'].max()

1682

In [15]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla K80


In [16]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 41.4 MB/s 
     |████████████████████████████████| 3.3 MB 36.6 MB/s 
     |████████████████████████████████| 61 kB 473 kB/s 
     |████████████████████████████████| 596 kB 51.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [17]:
df_new = data[['qtype', 'labels']].copy()
df_new.head(10)
df_new.to_excel("mapping.xlsx")

In [18]:
questions = df.question.values
ques_type = df.labels.values

In [19]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
import numpy as np
ques_combined = np.concatenate([df.question.values, df_test.question.values])
# print(qtypes_combined)
enc_comb = [tokenizer.encode(q, add_special_tokens=True) for q in ques_combined]
print("Maximum Length: ", max([len(sen) for sen in enc_comb]))

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Maximum Length:  187


In [20]:
import re
import string
# Create a function to tokenize a set of texts
#As max length of tokenised IDs is 187, we consider to do padding of length of tokens = 192
def preprocessing_for_bert(data):
   
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []
    
    
    for ques in data:
      ques = re.sub('[%s]' % re.escape(string.punctuation), '', str(ques)) 
      encoded_ques = tokenizer.encode_plus(
            text=ques,  
            add_special_tokens=True,        
            max_length=192,                  
            padding='max_length',         
            return_tensors='pt',           
            return_attention_mask=True      
            )
        
        # Add the outputs to the lists
      input_ids.append(encoded_ques.get('input_ids'))
      attention_masks.append(encoded_ques.get('attention_mask'))

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)


    return input_ids, attention_masks

In [21]:

import torch
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(questions, torch.tensor(ques_type), random_state=3000, test_size=0.1)


In [22]:
token_ids = list(preprocessing_for_bert([questions[0]])[0])
print('Original: ', questions[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
# print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

Original:  what periodical literature does delta air lines use as a moutpiece?
Token IDs:  [tensor([  101,  2054, 21802,  3906,  2515,  7160,  2250,  3210,  2224,  2004,
         1037,  9587,  4904, 11198,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [23]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_labels = Y_train
val_labels = Y_val
test_labels = torch.tensor(df_test.labels.values)

batch_size = 32
# print(type(Y_train))

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [24]:
import torch
import torch.nn as nn
from transformers import BertModel

class bertClassifier(nn.Module):
  def __init__(self, freeze_bert=False):
    super(bertClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    IN, H, OUT = 768,50,1683
    self.classifier = nn.Sequential(nn.Linear(IN, H),
                                    nn.ReLU(),
                                    nn.Dropout(0.3),
                                    nn.Linear(H,OUT))
    if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
                
  def forward(self, inp_ids, att_masks):
    output = self.bert(input_ids=inp_ids, attention_mask=att_masks)
    last_hidden = output[0][:,0,:]
    output = self.classifier(last_hidden)
    return output

In [25]:
from transformers import AdamW, get_linear_schedule_with_warmup
def initialize_model(epochs=4):
  bert_classifier = bertClassifier(freeze_bert=False)
  bert_classifier.to(device)
  optimizer = AdamW(bert_classifier.parameters(), lr=5e-5, eps=1e-8)
  total_steps = len(train_dataloader)*epochs
  scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0, num_training_steps=total_steps)
  return bert_classifier, optimizer, scheduler

In [26]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 100 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 100 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:

        # Load batch to GPU
      b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
      with torch.no_grad():
        logits = model(b_input_ids, b_attn_mask)

        # Compute loss
      loss = loss_fn(logits, b_labels)
      val_loss.append(loss.item())

        # Get the predictions
      preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
      accuracy = (preds == b_labels).cpu().numpy().mean() * 100
      val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [27]:
set_seed(60)    
bert_classifier, optimizer, scheduler = initialize_model(epochs=4)
train(bert_classifier, train_dataloader, val_dataloader, epochs=4, evaluation=True)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   100   |   6.557828   |     -      |     -     |  200.44  
   1    |   200   |   5.657633   |     -      |     -     |  197.37  
   1    |   300   |   5.113276   |     -      |     -     |  197.38  
   1    |   400   |   4.713807   |     -      |     -     |  197.46  
   1    |   500   |   4.484408   |     -      |     -     |  197.64  
   1    |   512   |   4.624813   |     -      |     -     |   23.73  
----------------------------------------------------------------------
   1    |    -    |   5.291912   |  4.193824  |   41.50   |  1057.56 
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   2    |   100   |   4.232405   |     -      |     -     |  199.

In [28]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    
    model.eval()
    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device)
         for t in batch)[:2]
        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [29]:
pred = bert_predict(bert_classifier, val_dataloader)

In [31]:
from sklearn.metrics import accuracy_score, precision_score
y_pred=np.argmax(pred, axis=1)
accuracy_score(val_labels, y_pred)

0.45668859649122806

In [32]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(val_labels, y_pred)

array([[[1823,    0],
        [   1,    0]],

       [[1704,   50],
        [  64,    6]],

       [[1597,   17],
        [   3,  207]],

       ...,

       [[1823,    0],
        [   1,    0]],

       [[1823,    0],
        [   1,    0]],

       [[1823,    0],
        [   1,    0]]])

In [33]:
# Concatenate the train set and the validation set
full_train_data = torch.utils.data.ConcatDataset([train_data, val_data])
full_train_sampler = RandomSampler(full_train_data)
full_train_dataloader = DataLoader(full_train_data, sampler=full_train_sampler, batch_size=32)

# Train the Bert Classifier on the entire training data
set_seed(75)
bert_classifier, optimizer, scheduler = initialize_model(epochs=7)
train(bert_classifier, full_train_dataloader, epochs=7)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   100   |   6.611119   |     -      |     -     |  199.73  
   1    |   200   |   5.605241   |     -      |     -     |  197.34  
   1    |   300   |   5.053405   |     -      |     -     |  197.41  
   1    |   400   |   4.666653   |     -      |     -     |  197.41  
   1    |   500   |   4.472999   |     -      |     -     |  197.44  
   1    |   569   |   4.334555   |     -      |     -     |  136.16  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   2    |   100   |   4.027191   |     -      |     -     |  199.68  
   2    |   200   |   3.955553   |     -      |     -     |  197.91  
   2    |   300   |   3.882928   |     -      |     -     |  197.8

In [34]:
test_inputs, test_masks = preprocessing_for_bert(df_test.question)

# Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks)
test_sample = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sample, batch_size=32)

In [35]:
# from sklearn.metrics import accuracy_score
probs = bert_predict(bert_classifier, test_dataloader)
# test_labels = torch.tensor()
y_pred=np.argmax(probs, axis=1)
accuracy_score(test_labels, y_pred)

0.5009857612267251

In [48]:
from sklearn.metrics import classification_report

label_names = np.asarray(df_test['labels'].unique())
# print(type(label_names))
# label_names.append(1683)
lab = np.append(label_names,[1683])
target_names=list(map(str,lab))

print(classification_report(test_labels, y_pred,target_names=target_names))

              precision    recall  f1-score   support

          21       0.23      0.53      0.32       164
           3       0.91      0.99      0.95       532
         106       0.61      0.88      0.72       277
         387       0.00      0.00      0.00         2
         605       0.86      0.94      0.90       502
        1036       0.00      0.00      0.00         7
           7       0.00      0.00      0.00         2
         265       0.00      0.00      0.00         3
          56       0.00      0.00      0.00         8
          41       0.33      0.33      0.33         6
          85       0.00      0.00      0.00         1
         349       0.00      0.00      0.00         1
          22       0.00      0.00      0.00         1
          40       0.18      0.37      0.24        78
         391       0.00      0.00      0.00         2
         145       0.08      0.15      0.11        13
         133       0.25      0.71      0.37        49
          44       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1